# Qwen

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm

In [ ]:
#FILENAMES
model_name = "Qwen/Qwen2.5-7B-Instruct"
filename="617points.csv"

#Load Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",      
    torch_dtype=torch.bfloat16  
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

#

## 1A. Binary IPV detection prompt

## 1B. Multi-label (TYPE) detection prompt

## 1C. IPV Subtype Detection

## 2. Generate Predictions

In [ ]:
def run_llm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0,   # deterministic
        do_sample=False
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text


In [ ]:
#testing on one
sample = make_binary_prompt(df.loc[0, "items"])
print(run_llm(sample))


In [ ]:
#proceed with all rows
binary_preds = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    prompt = make_binary_prompt(row["items"])
    result = run_llm(prompt)
    binary_preds.append(result)

df["Binary_Prediction"] = binary_preds


In [ ]:
#save to df.to_csv("llm_predictions.csv", index=False)
#next steps: Evaluate on your labeled data

# from sklearn.metrics import classification_report
# df["True_Binary"] = df[["Physical Abuse", "Emotional Abuse", "Sexual Abuse"]].any(axis=1)
# df["True_Binary"] = df["True_Binary"].astype(int)

# # Example parsing 'IPV' / 'NOT_IPV' from LLM output
# df["Pred_Binary"] = df["Binary_Label"].apply(lambda x: 1 if x and x.get("label") == "IPV" else 0)

# print(classification_report(df["True_Binary"], df["Pred_Binary"]))


## 3. Evaluation Pipeline